# Defining our own types

We have briefly seen a few examples of defining our own types in Julia. 

A type tells the computer how to interpret a block of memory.

A type is like a template for a box, that contains data.
A simple example is a "volume" type, that represents the volume of an object:

In [1]:
type Vol1
    value
end

In [2]:
V = Vol1(3)

Vol1(3)

In [3]:
V

Vol1(3)

In [4]:
typeof(V)

Vol1

`V` is an **instance** of the `Vol1` type.

I can extract its value:

In [6]:
V.value

3

Change the value:

In [7]:
V.value = 4

4

In [8]:
V2 = Vol1(5)

Vol1(5)

In [9]:
V

Vol1(4)

In [10]:
V2

Vol1(5)

Make a vector:

In [11]:
[Vol1(5), Vol1(6)]

2-element Array{Vol1,1}:
 Vol1(5)
 Vol1(6)

In [12]:
[Vol1(i) for i in 1:5]

5-element Array{Vol1,1}:
 Vol1(1)
 Vol1(2)
 Vol1(3)
 Vol1(4)
 Vol1(5)

In [13]:
map(Vol1, 1:5)

5-element Array{Vol1,1}:
 Vol1(1)
 Vol1(2)
 Vol1(3)
 Vol1(4)
 Vol1(5)

In [14]:
Vol1.(1:5)

5-element Array{Vol1,1}:
 Vol1(1)
 Vol1(2)
 Vol1(3)
 Vol1(4)
 Vol1(5)

We can show this nicely by overloading the `show` function on our type:

In [15]:
import Base.show

In [16]:
show

show (generic function with 212 methods)

In [17]:
show([1,2])

[1,2]

In [18]:
type Vol1a
    value
end

import Base: show

show(io::IO, V::Vol1a) = print(io, "Volume with value ", V.value)

show (generic function with 213 methods)

[We have defined a new type, since in Julia 0.5 the redefinition of `show` will not have any effect, since the previous version has been cached.]

In [19]:
V = Vol1a(3)

Volume with value 3

In [20]:
Vol1a.(1:5)

5-element Array{Vol1a,1}:
 Volume with value 1
 Volume with value 2
 Volume with value 3
 Volume with value 4
 Volume with value 5

We can define e.g. the sum of two volumes:

In [21]:
import Base: +

+(V1::Vol1a, V2::Vol1a) = Vol1a(V1.value + V2.value)

+ (generic function with 164 methods)

In [23]:
W = V + V

Volume with value 6

In [24]:
typeof(W)

Vol1a

But the following does not work:

In [25]:
V + 3

LoadError: MethodError: no method matching +(::Vol1a, ::Int64)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +([1m[31m::Complex{Bool}[0m, ::Real) at complex.jl:151
  +([1m[31m::Char[0m, ::Integer) at char.jl:40
  ...[0m

In [26]:
+(V::Vol1a, x::Int) = Vol1a(V.value + x)

+ (generic function with 165 methods)

In [27]:
V + 3

Volume with value 6

In [28]:
V + 3.1

LoadError: MethodError: no method matching +(::Vol1a, ::Float64)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +{T<:AbstractFloat}([1m[31m::Bool[0m, ::T<:AbstractFloat) at bool.jl:55
  +([1m[31m::Float64[0m, ::Float64) at float.jl:240
  ...[0m

In [29]:
+(V::Vol1a, x::Union{Int64, Float64}) = Vol1a(V.value + x)

+ (generic function with 166 methods)

In [30]:
V + 3.1

Volume with value 6.1

In [32]:
x = big(3.1)

3.100000000000000088817841970012523233890533447265625000000000000000000000000000

In [33]:
typeof(x)

BigFloat

In [35]:
precision(x)  # cf. Float64 that has 53 bits of precision in the mantissa (fractional part)

256

In [36]:
x

3.100000000000000088817841970012523233890533447265625000000000000000000000000000

In [37]:
isa(x, Float64)

false

In [38]:
big"3.1"

3.099999999999999999999999999999999999999999999999999999999999999999999999999986

In [39]:
parse(BigFloat, "3.1")

3.099999999999999999999999999999999999999999999999999999999999999999999999999986

In [ ]:
+(V::Vol1a, x::Union{Int64, Float64}) = Vol1a(V.value + x)

In [40]:
supertype(Int64)

Signed

In [41]:
supertype(Signed)

Integer

In [42]:
supertype(Float64)

AbstractFloat

In [43]:
supertype(AbstractFloat)

Real

In [44]:
supertype(Integer)

Real

In [45]:
supertype(Real)

Number

In [46]:
supertype(Number)

Any

In [47]:
+(V::Vol1a, x::Real) = Vol1a(V.value + x)

+ (generic function with 167 methods)

In [48]:
V + big(3.1)

Volume with value 6.100000000000000088817841970012523233890533447265625000000000000000000000000000

In [49]:
V + (0 + 1im)

LoadError: MethodError: no method matching +(::Vol1a, ::Complex{Int64})[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +([1m[31m::Bool[0m, ::Complex{T<:Real}) at complex.jl:143
  +(::Vol1a, [1m[31m::Int64[0m) at In[26]:1
  ...[0m

In [50]:
+(V::Vol1a, x::Number) = Vol1a(V.value + x)

+ (generic function with 168 methods)

In [52]:
V + (0 + 1im)

Volume with value 3 + 1im

In [53]:
+(V::Vol1a, x) = Vol1a(V.value + x)  # same as +(V::Vol1a, x::Any)

+ (generic function with 169 methods)

In [54]:
+(V::Vol1a, x::Complex) = Vol1a(V.value + real(x))

+ (generic function with 170 methods)

This is all equivalent to something like:

    function +(V, x)
        if typeof(V) == Vol1a && typeof(x) == Real
            return ...

        elseif typeof(V) == Vol1a && typeof(x) == Complex
            return ...
        
        end
    end
        

In [56]:
V

Volume with value 3

In [57]:
V + "hello"

LoadError: MethodError: no method matching +(::Int64, ::String)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +([1m[31m::Vol1a[0m, ::Any) at In[53]:1
  +{T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}}(::T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}, [1m[31m::T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}[0m) at int.jl:32
  ...[0m

**Exercise**: Define the sum of an integer and a string and then try this. (You will (in Julia 0.5) have to restart and define this before you do the sum.)

**Exercise**: Define `*` of two `Vol`s  and of a `Vol` and a number.

What we've just done is the epitome of multiple dispatch.
It's operator overloading "on steroids". 

## Restricting the type of fields of a type

There is a problem with our definition of the type `Vol1a`.

In [59]:
Vol1a("hello")

Volume with value hello

It doesn't make sense to have a string as a volume. So we should **restrict** which kinds of `value` are allowed, i.e. the **type** of `value`:

In [60]:
type Vol2
    value::Float64  # can only take values that are Float64s
end

In [61]:
Vol2(3.1)

Vol2(3.1)

In [62]:
Vol2(3)

Vol2(3.0)

In [63]:
Vol2("hello")

LoadError: MethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.

In [64]:
Vol2("3.1")

LoadError: MethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.

In [65]:
convert(::Type{Float64}, s::String) = parse(Float64, s)

LoadError: error in method definition: function Base.convert must be explicitly imported to be extended

In [66]:
import Base.convert

convert(::Type{Float64}, s::String) = parse(Float64, s)

convert (generic function with 600 methods)

In [67]:
Vol2("3.1")  # now it works

Vol2(3.1)

In [78]:
x = 2 // 7

2//7

In [79]:
typeof(x)

Rational{Int64}

In [83]:
x * x

4//49

In [82]:
y = Float64(x)
y * y

0.08163265306122448

In [84]:
Vol2(3//4) 

Vol2(0.75)

# Different types of volume: **parameteric types**

Now we can imagine that in different contexts, we could want integer volumes, or rational volumes, rather than `Vol`s which contain a floating-point number, e.g. for a 3D printer that makes everything out of cubes of the same size. 

We could define the following sequence of different types.

In [85]:
type Vol_Int
    value::Int
end

type Vol_Float
    value::Float64
end

type Vol_Rational
    value::Rational{Int64}
end

In [86]:
Vol_Int(3)

Vol_Int(3)

In [87]:
Vol_Int(3.1)

LoadError: InexactError()

In [89]:
V = Vol_Float(3.1)

Vol_Float(3.1)

In [91]:
V = Vol1(5)

Vol1(5)

In [92]:
V.value = "hello"

"hello"

But clearly this is the wrong way to do it, since we're repeating ourselves, and there is a strong principle not to do so (https://en.wikipedia.org/wiki/Don't_repeat_yourself).

Isn't there a more efficient way, where Julia itself can generate all of these different types?

What we would like to do is tell Julia that the **type** itself (here, `Int`, `Float64` or `Rational{Int64}`) 
is a **parameter** that we will specify. 

The syntax in Julia for this is to use curly braces (`{`, `}`) to specify such a **type parameter**:

In [93]:
type Vol3{T}   # T is a type parameter -- what we were looking for
    value
end

We can now pass in **any type** and `T` will be replaced by that type, creating a new type, e.g.

In [94]:
V = Vol3{Float64}(3.1)

Vol3{Float64}(3.1)

In [95]:
typeof(V)  

Vol3{Float64}

The result `Vol3{Float64}` is Julia's way of writing `Vol_Float64`.

In [96]:
V2 = Vol3{Int64}(4)  

Vol3{Int64}(4)

In [98]:
typeof(V2)

Vol3{Int64}

We see that the types of `V1` and `V2` are *different* (but related), and we have achieved what 
we wanted.

The type `Vol3` is called a **parametric type**, with **type parameter** `T`. Parameteric types may have several type parameters, as we have already seen with `Array`s:

In [99]:
a = [3, 4, 5]
typeof(a)

Array{Int64,1}

The type parameters here are `Int64`, which is itself a type, and the number `1`.

## Improving the solution

The problem with this solution is the following, which echos what happened at the start of the notebook:

In [100]:
V = Vol3{Int64}(3.1)

Vol3{Int64}(3.1)

In [101]:
typeof(V.value)

Float64

The type of the element (here, `Float64`) is disconnected from the type parameter (`Int64`). 
So we have not yet actually captured the pattern of `Vol2`,
which restricted the `value` field to be of the desired type.

We solve this be specifying the field to **also be of type `T`**, with the **same `T`**:

MAIN POINT OF THE NOTEBOOK:

In [102]:
type Vol4{T}
    value::T
end

For example,

In [105]:
V1 = Vol4{Int64}(3)

Vol4{Int64}(3)

In [106]:
V2 = Vol4{Int64}(3.0)

Vol4{Int64}(3)

In [107]:
V3 = Vol4{Float64}(3.0)

Vol4{Float64}(3.0)

In [108]:
typeof(V.value)

Int64

Now when we try to do 

In [109]:
Vol4{Int64}(3.1)

LoadError: InexactError()

If explicitly want to round 3.1 to 3, we can do it by calling explicitly the correct function:

In [110]:
floor(3.1)

3.0

In [111]:
floor(Int, 3.1)

3

In [112]:
convert(Int, 3.1)

LoadError: InexactError()

### Conversion subversion

Let's subvert Julia by changing how to convert an `Float64` to an `Int`:

In [113]:
import Base.convert

convert(::Type{Int64}, x::Float64) = floor(Int, x)

convert (generic function with 601 methods)

In [114]:
convert(Int64, 3.1)

3

In [116]:
type Vol4a{T}
    value::T
end

In [117]:
Vol4a{Int64}(3.1)

Vol4a{Int64}(3)

Julia throws an error, namely an `InexactError()`.
This means that we are trying to "force" the number 3.1 into a "smaller" type `Int64`, i.e. one in which it can't be represented.

However, now we seem to be repeating ourselves again: We know that `3.1` is of type `Float64`, and in fact Julia knows this too; so it seems redundant to have to specify it. Can't Julia just work it out? Indeed it can!:

In [ ]:
Vol2(3.1)

Here, Julia has **inferred** the type `T` from the "inside out". That is, it did some pattern matching to realise that `value::T` was **matched** if `T` was chosen to be `Float64`, and then propagated this same value of `T` **upwards** to the type parameter.

## More fields

**Exercise**: Define a `Point` type that represents a point in a 2D plane, with two fields, `x` and `y`. What are the options for this type, mirroring the types `Vol1` through `Vol4`?

## Summary

With parametric types, we have the following possibilities:

1. Julia converts (if possible) to the header type

2. Julia infers the header type from the inside (through the argument)


# Constructors

When we define a type, Julia also defines the **constructor functions** that we have been using above. These are functions with exactly the same name as the type.

They can be discovered using `methods`:

In [ ]:
methods(Vol1)

We see that Julia provides two default constructors.

For parametric types, it is a bit more complicated:

In [ ]:
methods(Vol4)

In [ ]:
methods(Vol4{Float64})

## Outer constructors

Julia allows us to provide our own constructor functions.
E.g.

In [121]:
Vol2(3.1)

Vol2(3.1)

In [122]:
Vol2("3.1")

Vol2(3.1)

Here, we have tried to provide a numeric string, which is not allowed, since the string is not a number. We can add a constructor to allow this:

In [123]:
methods(Vol2)

# 3 methods for generic function "(::Type)":
Vol2(value::Float64) at In[60]:2
Vol2(value) at In[60]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [124]:
Vol2(s::String) = Vol2(parse(Float64, s))

Vol2

In [125]:
methods(Vol2)

# 4 methods for generic function "(::Type)":
Vol2(value::Float64) at In[60]:2
Vol2(s::String) at In[124]:1
Vol2(value) at In[60]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [3]:
workspace()  # clears all variables

Restart the kernel here to clear the strange `convert` we defined.

In [1]:
Int64("31")

LoadError: MethodError: Cannot `convert` an object of type String to an object of type Int64
This may have arisen from a call to the constructor Int64(...),
since type constructors fall back to convert methods.

In [2]:
Int64(s::String) = parse(Int64, s)

Int64

In [3]:
Int64("31")

31

We have added a new constructor outside the type definition, so it is called an **outer constructor**.

## Constructors that impose a restriction: **inner constructors**

Now consider the following:

In [5]:
type Vol2
    value::Float64
end

In [6]:
Vol2(-1)

Vol2(-1.0)

Oops! A (naive) volume cannot be negative, but our constructors allow us to make a negative volume. To prevent this, we can define a constructor **within the type definition itself**, called **inner constructors**, that allows us to impose a restriction, or in general allows us to force objects to be constructed only in a certain way.

[In Julia, these are the **only methods** that may be defined inside the type definition. Unlike in object-oriented languages, methods **do not belong to types** in Julia; rather, they exist outside any particular type, and (multiple) dispatch is used instead.]

For example:

In [7]:
type Vol5
    value::Float64
    
    function Vol5(V) 
        if V < 0
            throw(ArgumentError("Volumes cannot be negative"))
        end
        
        new(V)   # `new` is the function that creates the object
    end
end

In [8]:
Vol5(3)

Vol5(3.0)

In [9]:
Vol5(-34)

LoadError: ArgumentError: Volumes cannot be negative

If we define an inner constructor, then Julia no longer defines the standard constructors; this is why defining an inner constructor gives us exclusive control over how our objects are created.

## Inner constructors for parametric types

# `type` vs `immutable`

So far we have been using `type`. But this allows the following:

In [10]:
V = Vol5(3)

Vol5(3.0)

In [11]:
V.value = -3

-3

In [ ]:
V

Oops! We have violated the restriction on negative volumes.
To get around this, we use `immutable` instead of `type`:

In [13]:
immutable Vol7   # just changed `type` to `immutable`
    value::Float64
    
    function Vol7(V) 
        if V < 0
            throw(ArgumentError("Volumes cannot be negative"))
        end
        
        new(V)
    end
end

In [14]:
V = Vol7(-3)

LoadError: ArgumentError: Volumes cannot be negative

In [15]:
V = Vol7(3)

Vol7(3.0)

In [16]:
V.value = -4

LoadError: type Vol7 is immutable

In [17]:
V

Vol7(3.0)

In [18]:
V = Vol7(2)

Vol7(2.0)

Have a value restricted to a range:

In [20]:
type MyValue
    v::Int64
end

In [21]:
x = MyValue(3)

MyValue(3)

In [22]:
function modify!(x::MyValue, i)
    if !(-10 <= i <= 10)
        throw(ArgumentError("i must be in range"))
    end
    
    x.v = i
end

modify! (generic function with 1 method)

In [23]:
x

MyValue(3)

In [27]:
x = MyValue(3)

MyValue(3)

In [28]:
modify!(x, 17)

LoadError: ArgumentError: i must be in range

In [29]:
x

MyValue(3)

In [30]:
modify!(x, 10)

10

In [31]:
x

MyValue(10)

## Performance gain with `immutable`s

The fact that an object is `immutable` also has performance implications, namely it allows the compiler to do optimizations to gain more performance.

An example is arrays: an `Array` of objects of an `immutable` type is stored directly in memory, whereas an `Array` of objets of a `type` are stored using pointers.

# Syntax changes in Julia 0.6

The syntax for parametric types will change in Julia 0.6.